In [1]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix


In [2]:
dom1 = []
dom2 = []
test_set = []
with open('data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("data/sample.csv")

with open('data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [3]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [4]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2149


In [5]:
data = pd.concat([dom1, dom2], ignore_index=True)

In [6]:
from sklearn.model_selection import train_test_split
data['model'].fillna(8, inplace=True)


X = data[['text','model']] 
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=data[['model', 'label']], random_state=42)

In [7]:
y_train

9936     0
25220    0
24497    0
9566     1
18884    0
        ..
29451    0
4594     1
29348    0
26916    0
28576    0
Name: label, Length: 27519, dtype: int64

In [8]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

print(X_train['model'].value_counts(normalize=True))
print(X_test['model'].value_counts(normalize=True))

label
0    0.654094
1    0.345906
Name: proportion, dtype: float64
label
0    0.65407
1    0.34593
Name: proportion, dtype: float64
model
8.0    0.629347
0.0    0.068716
1.0    0.068534
3.0    0.068534
2.0    0.067989
6.0    0.051274
4.0    0.022930
5.0    0.022675
Name: proportion, dtype: float64
model
8.0    0.629360
0.0    0.068750
3.0    0.068605
1.0    0.068459
2.0    0.068023
6.0    0.051163
4.0    0.022965
5.0    0.022674
Name: proportion, dtype: float64


In [9]:
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np


X = X_train['text'].tolist()
X1 =X_test['text'].tolist()
max_length = max([len(seq) for seq in X])
max_length1 = max([len(seq) for seq in X1])
word2vec = Word2Vec(X, vector_size=300, window=10, min_count=1, workers=4)
word2vec1 = Word2Vec(X1, vector_size=300, window=10, min_count=1, workers=4)

def get_text_vector(text, word2vec_model):
    vector_list = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv.index_to_key]
    if len(vector_list) == 0:
        return np.zeros(word2vec.vector_size)
    return np.mean(vector_list, axis=0)

X_train = np.array([get_text_vector(text, word2vec) for text in X])
X_test = np.array([get_text_vector(text, word2vec1) for text in X1])

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks





smote = SMOTE(sampling_strategy=0.7)
tomek = TomekLinks(sampling_strategy='majority')


pipeline = Pipeline(steps=[('o', smote), ('u', tomek)])


X_train, y_train = pipeline.fit_resample(X_train, y_train)





In [10]:
print(y_train.value_counts(normalize=True))

label
0    0.58554
1    0.41446
Name: proportion, dtype: float64


In [11]:
y_train

0        0
1        0
2        0
3        1
4        0
        ..
30396    1
30397    1
30398    1
30399    1
30400    1
Name: label, Length: 30401, dtype: int64

In [12]:
print(len(X_train))
print(len(y_train))


30401
30401


In [13]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to('cuda' if torch.cuda.is_available() else 'cpu')


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

batch_size = 64  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)




In [14]:
X_train_tensor

tensor([[ 0.2744,  0.0799, -0.4938,  ..., -0.1175,  0.0989, -0.1265],
        [-0.2202,  0.3285, -0.1467,  ..., -0.1497, -0.1044,  0.0025],
        [-0.1474,  0.2807, -0.0995,  ..., -0.3048,  0.0586,  0.0586],
        ...,
        [-0.0128,  0.1453, -0.2720,  ..., -0.1786,  0.0658,  0.1536],
        [-0.0091,  0.5468, -0.1467,  ..., -0.1265, -0.1568,  0.1994],
        [-0.0825,  0.4063, -0.1860,  ..., -0.3794, -0.1103, -0.0289]])

In [15]:
y_train_tensor

tensor([0, 0, 0,  ..., 1, 1, 1])

In [16]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)



torch.Size([30401, 300])
torch.Size([30401])


In [17]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import TensorDataset, DataLoader

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64  
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [41]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 150, output_dim) # Assuming the length of sequences is 300, after pooling it will be 150
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change the shape to (batch_size, input_dim, seq_len)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.reshape(x.size(0), -1)  # Flatten the tensor using reshape instead of view
        x = self.fc1(x)
        return self.sigmoid(x)


input_dim = 300
output_dim = 1

model = CNNModel(input_dim, output_dim)



In [39]:


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)


In [37]:
# 假设 train_loader 和 test_loader 已被定义
for batch in train_loader:
    text, labels = batch
    print(text.size())  # 这应该给你类似 (batch_size, seq_len, input_dim) 的输出
    break


torch.Size([64, 300])


In [42]:
import torch.nn.functional as F
def train(model, iterator, optimizer, criterion):
    model.train()
    total_loss = 0
    total_samples = 0
    for batch in iterator:
        text, labels = batch
        optimizer.zero_grad()
        text = text.unsqueeze(2)  # Adding a channel dimension

        predictions = model(text).squeeze(1)
        loss = criterion(predictions, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_samples += labels.size(0) 

    return total_loss / total_samples  

def evaluate(model, iterator, criterion):
    model.eval()
    total_loss = 0
    total_samples = 0
    with torch.no_grad():
        for batch in iterator:
            text, labels = batch  
            text = text.unsqueeze(2)  # Adding a channel dimension
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, labels.float())
            total_loss += loss.item()
            total_samples += labels.size(0) 

    return total_loss / total_samples  


N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_loss = evaluate(model, test_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTest Loss: {test_loss:.3f}')
    best_test_loss = float('inf')
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'model_weights.pth')




Epoch: 01
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 02
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 03
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 04
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 05
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 06
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 07
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 08
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 09
	Train Loss: 0.011
	Test Loss: 0.012
Epoch: 10
	Train Loss: 0.011
	Test Loss: 0.012


In [43]:

print(model)


CNNModel(
  (conv1): Conv1d(1, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=19200, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [44]:
print(model.state_dict())


OrderedDict([('conv1.weight', tensor([[[ 0.3129,  0.4963, -0.3704]],

        [[-0.2359, -0.0741, -0.5163]],

        [[ 0.3737,  0.0266, -0.0522]],

        [[-0.0656,  0.2748,  0.0875]],

        [[-0.4687, -0.3957, -0.0183]],

        [[-0.3248, -0.4346, -0.5567]],

        [[-0.1224, -0.4323,  0.1989]],

        [[ 0.5625, -0.5348, -0.3639]],

        [[ 0.4276, -0.3076, -0.4434]],

        [[ 0.1432,  0.1441,  0.3495]],

        [[-0.3279, -0.0319,  0.1598]],

        [[ 0.0876, -0.3682,  0.2386]],

        [[-0.2241, -0.0266, -0.3116]],

        [[-0.2189,  0.0779,  0.3427]],

        [[-0.1317, -0.3078, -0.2342]],

        [[-0.4304, -0.0047,  0.5203]],

        [[ 0.4373, -0.5699, -0.2202]],

        [[-0.0079,  0.0174, -0.4909]],

        [[ 0.0458, -0.5333,  0.3216]],

        [[-0.4678, -0.3653,  0.4644]],

        [[-0.3523, -0.0725, -0.3164]],

        [[-0.0502, -0.4994, -0.1163]],

        [[-0.0274,  0.5268,  0.2163]],

        [[ 0.2056, -0.2588, -0.3360]],

        [[

In [45]:
X2 =test_set['text'].tolist()
max_length2 = max([len(seq) for seq in X2])
word2vec2 = Word2Vec(X2, vector_size=300, window=10, min_count=1, workers=4)
test_set = np.array([get_text_vector(text, word2vec2) for text in X2])
test_set_tensor = torch.tensor(test_set, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

In [59]:
test_set_tensor

tensor([[ 9.1950e-03,  3.8779e-01,  5.8890e-02,  ...,  2.4433e-02,
          2.3045e-01, -1.9151e-01],
        [-7.1910e-02,  2.7059e-01, -7.3820e-02,  ...,  7.1633e-03,
          1.0798e-01, -2.3288e-01],
        [ 2.2877e-02,  3.7363e-01,  7.3490e-02,  ...,  3.2050e-02,
          1.7525e-01, -1.0136e-01],
        ...,
        [ 4.1560e-02,  3.9480e-01,  1.0509e-01,  ...,  3.4753e-02,
          1.9286e-01, -8.0040e-02],
        [ 2.0246e-02,  4.0686e-01,  7.6063e-02,  ...,  2.7383e-02,
          2.4095e-01, -1.8053e-01],
        [-2.4317e-02,  3.4818e-01,  2.7681e-04,  ...,  1.9132e-02,
          1.7545e-01, -2.0126e-01]])

In [62]:
# 仅作示例，你需要使用你的数据和模型
for data in test_set:  # Assuming train_loader is your training data loader
    print(data.shape)
    break


(300,)


In [110]:
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

# Assuming test_set_tensor is a tensor containing your test data
predictions = []

with torch.no_grad():
    for index, text in enumerate(test_set_tensor):
        try:
            text = text.unsqueeze(0)
            text = text.unsqueeze(0) # Adding a batch dimension
            text = text.permute(0, 2, 1)  # Adjusting the order of dimensions
            predictions_tensor = model(text)
            predicted_class = (predictions_tensor > 0.575).float()
            label = int(predicted_class.item())
            predictions.append({"id": index, "label": label})
        except Exception as e:
            print(f"An error occurred while processing sample at index {index}: {e}")




# Convert predictions to a pandas DataFrame
predictions_df = pd.DataFrame(predictions)


In [111]:
predictions[:10]

[{'id': 0, 'label': 0},
 {'id': 1, 'label': 1},
 {'id': 2, 'label': 0},
 {'id': 3, 'label': 0},
 {'id': 4, 'label': 0},
 {'id': 5, 'label': 0},
 {'id': 6, 'label': 0},
 {'id': 7, 'label': 0},
 {'id': 8, 'label': 1},
 {'id': 9, 'label': 0}]

In [112]:
import csv
output_csv_file = 'predictions.csv'
with open(output_csv_file, 'w', newline='') as csvfile:
    fieldnames = ['id', 'class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for prediction in predictions:
        writer.writerow({'id': prediction['id'], 'class': prediction['label']})

print(f'Predictions saved to {output_csv_file}')

Predictions saved to predictions.csv


In [113]:
result = pd.read_csv('predictions.csv')

In [114]:
result[result['class'] == 1]

,id,class
1,1,1
8,8,1
10,10,1
12,12,1
13,13,1
...,...,...
987,987,1
990,990,1
993,993,1
995,995,1


In [115]:
result.to_csv('cnn_word2.csv', index=False)